In [11]:
import h5py
import matplotlib.pyplot as plt
import numpy as np
import os

import pandas as pd

import torch
import torch.autograd as autograd
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from torch.utils.data import Dataset

import wfdb

%matplotlib inline

In [12]:
class SleepDataset(Dataset):
    """Physionet 2018 dataset."""

    def __init__(self, records_file, root_dir, s, f):
        """
        Args:
            records_file (string): Path to the records file.
            root_dir (string): Directory with all the signals.

        """
        self.landmarks_frame = pd.read_csv(records_file)[s:f]
        self.root_dir = root_dir

    def __len__(self):
        return len(self.landmarks_frame)

    def __getitem__(self, idx):
        folder_name = os.path.join(self.root_dir,
                                self.landmarks_frame.iloc[idx, 0])
        file_name = self.landmarks_frame.iloc[idx, 0]
        signals = wfdb.rdrecord(os.path.join(folder_name, file_name[:-1]))
        arousals = h5py.File(os.path.join(folder_name, file_name[:-1] + '-arousal.mat'), 'r')

        sample = torch.FloatTensor(
            signals.p_signal #[:1000]
        ), torch.LongTensor(
            arousals['data']['arousals'].value.clip(min=0).ravel().astype(int)
        )

        return sample

a = SleepDataset('../data/training/RECORDS', '../data/training/', 0, 2)
print(len(a))
for i in range(2):
    sample = a[i]


2


In [13]:
sample[0]


  -43.0000    10.0000   -13.0000  ...     28.0000    93.5653     0.1870
  -76.0000    12.0000   -29.0000  ...     58.0000    93.5653     0.2760
 -102.0000     9.0000   -43.0000  ...     51.0000    93.5653     0.3570
              ...                  ⋱                 ...               
    0.0000     0.0000     0.0000  ...     -2.0000     0.4959    -0.1740
    0.0000     0.0000     0.0000  ...      0.0000     0.4959    -0.0390
    0.0000     0.0000     0.0000  ...      0.0000     0.4959    -0.2450
[torch.FloatTensor of size 5992000x13]

In [5]:
#print(sample)
# print(outputs)
sample[1]


 0
 0
 0
⋮ 
 0
 0
 0
[torch.LongTensor of size 5992000]

In [6]:
# Multilayer LSTM based classifier taking in 200 dimensional fixed time series inputs
class LSTMClassifier(nn.Module):

    def __init__(self, in_dim, hidden_dim, num_layers, dropout, bidirectional, num_classes, batch_size):
        super(LSTMClassifier, self).__init__()
        self.arch = 'lstm'
        self.hidden_dim = hidden_dim
        self.batch_size = batch_size
        self.num_dir = 2 if bidirectional else 1
        self.num_layers = num_layers

        self.lstm = nn.LSTM(
                input_size=in_dim,
                hidden_size=hidden_dim,
                num_layers=num_layers,
                dropout=dropout,
                bidirectional=bidirectional
            )

        self.hidden2label = nn.Sequential(
            nn.Linear(hidden_dim*self.num_dir, hidden_dim),
            nn.ReLU(True),
            nn.Dropout(),
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(True),
            nn.Dropout(),
            nn.Linear(hidden_dim, num_classes),
        )

        # self.hidden = self.init_hidden()

    def init_hidden(self):
        if cuda:
            h0 = Variable(torch.zeros(self.num_layers*self.num_dir, self.batch_size, self.hidden_dim).cuda())
            c0 = Variable(torch.zeros(self.num_layers*self.num_dir, self.batch_size, self.hidden_dim).cuda())
        else:
            h0 = Variable(torch.zeros(self.num_layers*self.num_dir, self.batch_size, self.hidden_dim))
            c0 = Variable(torch.zeros(self.num_layers*self.num_dir, self.batch_size, self.hidden_dim))
        return (h0, c0)

    def forward(self, x): # x is (batch_size, 1, 200), permute to (200, batch_size, 1)
        x = x.permute(2, 0, 1)
        # See: https://discuss.pytorch.org/t/solved-why-we-need-to-detach-variable-which-contains-hidden-representation/1426/2
        lstm_out, (h, c) = self.lstm(x, self.init_hidden())
        y  = self.hidden2label(lstm_out[-1])
        return y

In [20]:
batch_size = 16
n_iters = 3000
num_epochs = 2 # n_iters / (len(train_dataset) / batch_size)
num_epochs = int(num_epochs)

train_dataset = SleepDataset('../data/training/RECORDS', '../data/training/', 0, 2)
test_dataset = SleepDataset('../data/training/RECORDS', '../data/training/', 2, 4)

train_loader = torch.utils.data.DataLoader(dataset=train_dataset, 
                                           batch_size=batch_size, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset, 
                                          batch_size=batch_size, 
                                          shuffle=False)

'''
STEP 3: CREATE MODEL CLASS
'''

class LSTMTagger(nn.Module):

    def __init__(self, embedding_dim, hidden_dim, vocab_size, tagset_size):
        super(LSTMTagger, self).__init__()
        self.hidden_dim = hidden_dim

#         self.word_embeddings = nn.Embedding(vocab_size, embedding_dim)

        # The LSTM takes word embeddings as inputs, and outputs hidden states
        # with dimensionality hidden_dim.
        self.lstm = nn.LSTM(embedding_dim, hidden_dim)

        # The linear layer that maps from hidden state space to tag space
        self.hidden2tag = nn.Linear(hidden_dim, tagset_size)
        self.hidden = self.init_hidden()

    def init_hidden(self):
        # Before we've done anything, we dont have any hidden state.
        # Refer to the Pytorch documentation to see exactly
        # why they have this dimensionality.
        # The axes semantics are (num_layers, minibatch_size, hidden_dim)
        return (autograd.Variable(torch.zeros(1, 1, self.hidden_dim)),
                autograd.Variable(torch.zeros(1, 1, self.hidden_dim)))

    def forward(self, sentence):
#         embeds = self.word_embeddings(sentence)
        lstm_out, self.hidden = self.lstm(
            sentence.view(len(sentence), 1, -1), self.hidden)
        tag_space = self.hidden2tag(lstm_out.view(len(sentence), -1))
        tag_scores = F.log_softmax(tag_space, dim=1)
        return tag_scores

'''
STEP 4: INSTANTIATE MODEL CLASS
'''
input_dim = 13
hidden_dim = 2
layer_dim = 3  # ONLY CHANGE IS HERE FROM ONE LAYER TO TWO LAYER
output_dim = 2

model = LSTMTagger(input_dim, hidden_dim, layer_dim, output_dim)

    
'''
STEP 5: INSTANTIATE LOSS CLASS
'''
criterion = nn.NLLLoss()

'''
STEP 6: INSTANTIATE OPTIMIZER CLASS
'''
learning_rate = 0.01

optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)  

'''
STEP 7: TRAIN THE MODEL
'''

# Number of steps to unroll
seq_dim = 5992000

iter = 0
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        # Load images as Variable
        #######################
        #  USE GPU FOR MODEL  #
        #######################

        images = Variable(images)
        labels = Variable(labels)
            
        # Clear gradients w.r.t. parameters
        optimizer.zero_grad()
        
        # Forward pass to get output/logits
        # outputs.size() --> 100, 10
        outputs = model(images)
        print(outputs.size(), labels.size())
        # Calculate Loss: softmax --> cross entropy loss
        loss = criterion(outputs.view(-1, 2), labels.view(-1))
        
        # Getting gradients w.r.t. parameters
        loss.backward()
        
        # Updating parameters
        optimizer.step()
        
        iter += 1
        
        if iter % 500 == 1:
            # Calculate Accuracy         
            correct = 0
            total = 0
            # Iterate through test dataset
            for images, labels in test_loader:
                labels = labels.view(-1)
                images = Variable(images)
                labels
                # Forward pass only to get logits/output
                outputs = model(images)
                
                # Get predictions from the maximum value
                _, predicted = torch.max(outputs.data, 1)
                
                # Total number of labels
                total += labels.size(1)
                
                # Total correct predictions
                #######################

                correct += (predicted == labels).sum()
            
            accuracy = 100 * correct / total
            
            # Print Loss
            print('Iteration: {}. Loss: {}. Accuracy: {}'.format(iter, loss.data[0], accuracy))

RuntimeError: inconsistent tensor sizes at /pytorch/torch/lib/TH/generic/THTensorMath.c:2709

In [ ]:
labels = labels.view(-1)
#######################
#  USE GPU FOR MODEL  #
#######################


# Forward pass only to get logits/output
outputs = model(images)

# Get predictions from the maximum value
_, predicted = torch.max(outputs.view(-1, 2).data, 1)

# Total number of labels
total += labels.size(0)

# Total correct predictions
#######################
#  USE GPU FOR MODEL  #
#######################
if torch.cuda.is_available():
    correct += (predicted.cpu() == labels.cpu()).sum()
else:
    correct += (predicted == labels).sum()

In [ ]:
torch.max(outputs.view(-1, 2).data, 1)